# Capstone Project for the IBM Data Science Professional Certificate 
### Analysing locations for a clothing Store in New York evaluated with a criteria matrix


This Notebook is for the capstone project of the 9-course specialisation in data science of IBM 
https://www.coursera.org/professional-certificates/ibm-data-science
This project will solve a problem by applying data science methods with location data fetched from FourSquare API. The Notebook will guide you through the whole process of solving the business problem.

### Business Problem 

#### 1. Description of the Project: 

A Canadian Investor who recently made a fortune with an investment in a Clothing Store in Toronto wants to repeat his idea in New York. New York is one of America brightest Citys and among the worlds major commercial, financial and cultural centers. Who ever has come to New York is under the spell of the atmosphere and the vibrant life of this city. Its iconic sites include skyscrapers such as the Empire State building and the Broadway theatre located at Times Square, it is for sure a touristic hotspot in North America. With a expertly planned transport system which can take you everywhere in the city and booming business at every corner this city offers a great opportunity for the investor to open a successful clothing store but as being foreigner, he has not much idea of the structure of the gigantic City, therefore he needs some help with  his ideas in his mind, so he pointed out the following criteria for the location to be successful:

1. It should be one of the most crowded boroughs with high employment rate and good average income, because the Brand is pretty expansive and the investor wants not only tourists to buy in the store he also prefers regular customers.

2. As close to the city center or other touristic hotspots to benefit from walk-in customers. Approximately 20 Minutes walking distance to the center of the borough. If possible far away from other clothing stores.

3. A location close to Italian Restaurants is preferred, because of the origin of his brand and it could be possible that people stop by and buy some clothe before or after dinner or just for window shopping.

4. As close as possible to Hotels, because guests of the city Hotels are more likely to buy clothes nearby and guarantee for more walk-in customers.

5. A very important point for the Investor is a low crime rate to feel safe, also because he wants to rent a apartment close by the store. 



#### 1. Business Understanding

The Project seems very clear to find the perfect location for a italian brand clothing store in New York City. For the best result the analytical solution to the business problem is to quantify and evaluate the thoguhts of the client to fullfill his requirements completly. For evaluating his criteria we will create a features weight matrix to express the investors desires in a scientific way. Therefore we need to communicate again to weight his interests properly. 

#### 2. Analytical Approach 

First of all we need to quantify and evaluate the criteria given by the investor. 
To quantify his wishes we would express 20 minutes walking distance in 1300 meters  etc..
As mentioned before the criteria Matrix plays a crucial role in this project to weight the interest of the client properly we will evaluate his desires with the criteria matrix. Normalize the features and multiplay it with our customer data frame to add a extra column with the weighted result which gives us an indication of the best neighborhood.

For the second way we will solve this project with the unsupervised machiene learning method k-means to create clusters which will show us reasonable neighborhoods  to fit the interest of the client and to compare similarity between these neighborhoods. So we can support the previously result with a folium map 


#### 3. Data requirements 

To ensure the best location for the store i decided to add some more complexity to the standard course problem. As you can see from the criteria given by the investor we need some extra data like socioeconomic information and crime rate data to perform this analysis. For the socio-economic data we will build a data frame with all necessary information inclusive the crime rate.
The venue data will be fetched with the Foursquare Api and sorted by frequency. We will filter all the relevant venues. 

We will try to use representive data from official sources.





#### 4.  Data collection


-  For the Neigborhoods and the Population I found a list from Wikipedia: https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City 

- For the Neighborhoods I found another official source: 
https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas-NTA-/cpf4-rkhq

-  and also a more detailed table of the Population by neighbourhood from the same source: https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Neighborhood-Tabulatio/swpk-hqdp/data

- The Crime Data: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-Year-To-Date-/5uac-w243

- The Income table i got from:  https://ny.curbed.com/2017/8/4/16099252/new-york-neighborhood-affordability and another table from: https://data.cccnewyork.org/data/table/66/median-incomes#66/107/40/a/a

- Some data to the unemployment by neighborhoods was pretty dificult to find so i leave it for the moment. I found a source with unemployment by Zip Codes: http://zipatlas.com/us/ny/new-york/zip-code-comparison/unemployment-rate.htm
 

- For the venue data we will fetche the data with the Foursqaure Api.

- with the geocoder we can transfrom lat and lon to zip codes and reverse to match the data properly.


#### 5. Data understanding

Finding suitable data for this project was more difficult than it seems to be. Especially datasets to the crime rate and unemployment rate by neighborhoods or zipcodes was difficult to find. 

#### 6. Data preparation

We will try to build a clean dataframe with all the socio-economic data sets included the crime rate. 

The second dataframe will be the neighborhood venue data. Therefore we will fetch the venue data of the neighborhoods with the foursquare api with a function than cleaning, wrangling and changing the data to get only necessary columns. Afterwoards we will use one hot encoding for al venues of the df. The last step is to sort them by frequency of each neighborhood.

The next step is to build the features wheight criteria matrix, at this point we will only deal with data which is relevant to the customer. This means presenting the client a analysation which includes lakes, parks, chinese restaurant etc. is irrelevant to his business problem and guides us to a bad result. We will weight the prerequisites of the Investor in a scale from 1 to 10 in a manner of importance. 

Another important factor is normalization to bring our intrested features to a comparable grade. We will normalize our feature weight matrix to the numeric value between 0 and 1. For example the highest Income will be converted to 1, the highest frequency of hotels will be converted to 1 etc. so we can ensure that each feature has the same weight as the others.

Now we can multiply the criteria matrix with the customer data frame and add the total score column. The total score will give us insights of the perfect location

The next method is k-means analysation to cluster the best neighborhoods in accordance to the customer requirements this supports our previously result with a folium cluster map

In [4]:
# Installing and importing the necessary tools:

'''!pip install geopy 
!pip install folium
!pip install geopandas
!pip install descartes
!pip install xlrd
!pip install lxml
!pip install seaborn
!pip install matplotlib==3.1.2
!pip install matplotlib-venn'''

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import seaborn as sns 
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import xlrd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib import style
plt.rcParams['figure.dpi'] = 200
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import geopandas
import descartes
from bs4 import BeautifulSoup
import csv
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print("Libraries imported")

Libraries imported
